In [135]:
#pip install pandas

In [136]:
#pip install matplotlib

In [137]:
#pip install scipy

In [138]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [139]:
# STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
# STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  
#STARTINGCSV2025=pd.read_csv('NHL2526seasonplayerstats.csv')  
STARTINGCSV2025=pd.read_csv('NewPlayerStats2526.csv')  

STARTINGCSV=STARTINGCSV2025#pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameId'])

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
0,8477987,8,{'default': 'R. Donato'},C,0,0,0,0,0,3,0,1,0.000000,16:00,0,23,0,0,Ryan,Donato,CHI,2025020001,0
20,8480185,27,{'default': 'E. Luostarinen'},C,0,0,0,-1,0,3,0,2,0.000000,19:39,1,27,3,0,Eetu,Luostarinen,FLA,2025020001,1
21,8473419,63,{'default': 'B. Marchand'},L,0,0,0,-1,2,1,0,4,0.000000,20:10,0,27,1,0,Brad,Marchand,FLA,2025020001,1
22,8480003,70,{'default': 'J. Boqvist'},C,1,0,1,1,0,4,0,1,0.500000,10:38,0,21,0,0,Jesper,Boqvist,FLA,2025020001,1
23,8479316,71,{'default': 'L. Kunin'},C,0,0,0,1,2,0,0,3,0.600000,08:09,0,16,0,0,Luke,Kunin,FLA,2025020001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,8477951,8,{'default': 'N. Schmaltz'},C,0,1,1,1,0,1,0,6,0.000000,19:52,0,19,1,0,Nick,Schmaltz,UTA,2025020060,1
2137,8479343,9,{'default': 'C. Keller'},R,0,0,0,1,2,0,0,4,0.000000,20:23,0,20,2,0,Clayton,Keller,UTA,2025020060,1
2138,8482699,11,{'default': 'D. Guenther'},R,0,0,0,1,0,3,0,3,0.000000,16:54,0,17,0,0,Dylan,Guenther,UTA,2025020060,1
2140,8480855,22,{'default': 'J. McBain'},C,0,0,0,1,7,2,0,1,0.285714,14:17,1,18,0,0,Jack,McBain,UTA,2025020060,1


In [140]:
# #https://medium.com/@alexandre.tkint/send-pandas-data-frame-via-email-in-python-c346314321fa


# from email.mime.application import MIMEApplication
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# import smtplib
# def send_email(send_to, subject, df):
#     send_from = "dummy@gmail.com"
#     password = "your_password"
#     message = """\
#     <p><strong>This is a test email&nbsp;</strong></p>
#     <p><br></p>
#     <p><strong>Greetings&nbsp;</strong><br><strong>Alexandre&nbsp;    </strong></p>
#     """
# for receiver in send_to:
#         multipart = MIMEMultipart()
#         multipart["From"] = send_from
#         multipart["To"] = receiver
#         multipart["Subject"] = subject  
#         attachment = MIMEApplication(df.to_csv())
#         attachment["Content-Disposition"] = 'attachment; filename=" {}"'.format(f"{subject}.csv")
#         multipart.attach(attachment)
#         multipart.attach(MIMEText(message, "html"))
#         server = smtplib.SMTP("smtp.gmail.com", 587)
#         server.starttls()
#         server.login(multipart["From"], password)
#         server.sendmail(multipart["From"], multipart["To"], multipart.as_string())
#         server.quit()

In [141]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Colorado,Avalanche,COL
1,Carolina,Hurricanes,CAR
2,Dallas,Stars,DAL
3,Detroit,Red Wings,DET
4,Boston,Bruins,BOS
5,Montréal,Canadiens,MTL
6,Washington,Capitals,WSH
7,Vegas,Golden Knights,VGK
8,Florida,Panthers,FLA
9,Edmonton,Oilers,EDM


In [142]:
STARTINGCSV.head()

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
0,8477987,8,{'default': 'R. Donato'},C,0,0,0,0,0,3,0,1,0.000000,16:00,0,23,0,0,Ryan,Donato,CHI,2025020001,0
1,8477450,16,{'default': 'J. Dickinson'},C,0,0,0,0,0,0,0,1,0.578947,14:14,0,25,0,0,Jason,Dickinson,CHI,2025020001,0
2,8473422,17,{'default': 'N. Foligno'},L,0,0,0,-1,5,2,0,0,0.625000,11:48,0,21,2,0,Nick,Foligno,CHI,2025020001,0
3,8483450,20,{'default': 'R. Greene'},C,0,0,0,-1,0,0,0,0,0.250000,08:17,0,15,0,0,Ryan,Greene,CHI,2025020001,0
4,8478043,24,{'default': 'S. Lafferty'},C,0,0,0,-1,0,1,0,0,0.000000,08:07,0,13,0,0,Sam,Lafferty,CHI,2025020001,0


In [143]:
playerStats=STARTINGCSV[['Team','playerId','firstName','lastName','gameId','homeRoadFlag','sog','toi']]
playerStats['toi']=round(pd.to_numeric(playerStats['toi'].str.slice(0,2).str.replace(':', '', regex=False)
                                 )+(pd.to_numeric(playerStats['toi'].str.slice(3, 5))/60),2) #min+sec in decimal base not base60

playerStats['homeRoadFlag'] = np.where(playerStats['homeRoadFlag'] =='H', 1, 0)

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('Team').GameNumber.transform(np.max)
playerStats['HomeGames'] = playerStats.groupby(['playerId']).homeRoadFlag.transform(np.sum)
playerStats['AwayGames']=playerStats['GP']-playerStats['HomeGames']
playerStats['TotalShotsSzn']=playerStats.groupby(['playerId']).sog.transform(np.sum)
playerStats['SeasonAvgSOG']= round(playerStats['TotalShotsSzn']/playerStats['GP'],2)
#playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['sog']))

playerStats['AvgShotsSznByLoc']=playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean)

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)
playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).sog.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)
playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).sog.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)

#playerStats['AvgShotsAgainstThisOpp']=playerStats.groupby(['playerId','opponentAbbrev']).sog.transform(np.mean)

# homeawaystats=playerStats[['playerId','opponentAbbrev','gameDate','homeRoadFlag','sog']]
#playerStats['totalshotshomevaway']=playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.sum)
playerStats['avgshotshomevaway']=round(playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean),2)

# homeawaystats['GPHomeAwayvsOpp'] = homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag'])['gameDate'].cumcount(ascending=False)+1
# homeawaystats['GPHomeAwayvsOpp'] = homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag']).GPHomeAwayvsOpp.transform(np.max)

# playerStats=playerStats.merge(homeawaystats[['playerId','opponentAbbrev','homeRoadFlag','GPHomeAwayvsOpp','avgshotsvsOppAtLoc','totalshotsvsOppAtLoc']],on=['playerId','opponentAbbrev','homeRoadFlag'],how='left')
# playerStats['avgshotsvsOppAtLoc']=np.where(playerStats['avgshotsvsOppAtLoc'].isna(), 0, playerStats['avgshotsvsOppAtLoc'])
# playerStats['totalshotsvsOppAtLoc']=np.where(playerStats['totalshotsvsOppAtLoc'].isna(), 0, playerStats['totalshotsvsOppAtLoc'])


playerStats

,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway
0,CHI,8477987,Ryan,Donato,2025020001,0,1,16.00,5,5,0,5,12,2.40,2.40,1,2.4,1,1.2,2.40
1,CHI,8477450,Jason,Dickinson,2025020001,0,1,14.23,4,5,0,5,6,1.20,1.50,1,1.2,1,0.6,1.50
2,CHI,8473422,Nick,Foligno,2025020001,0,0,11.80,4,5,0,5,4,0.80,1.00,1,0.8,1,0.4,1.00
3,CHI,8483450,Ryan,Greene,2025020001,0,0,8.28,5,5,0,5,1,0.20,0.20,1,0.2,1,0.1,0.20
4,CHI,8478043,Sam,Lafferty,2025020001,0,0,8.12,2,5,0,5,0,0.00,0.00,1,0.0,1,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,UTA,8478507,John,Marino,2025020060,0,1,19.33,1,4,0,4,3,0.75,0.75,1,0.6,1,0.3,0.75
2156,UTA,8484386,Dmitri,Simashev,2025020060,0,1,14.28,1,4,0,4,6,1.50,1.50,1,1.2,1,0.6,1.50
2157,UTA,8474013,Ian,Cole,2025020060,0,0,16.53,1,4,0,4,1,0.25,0.25,1,0.2,1,0.1,0.25
2158,UTA,8477220,Nate,Schmidt,2025020060,0,1,22.48,1,4,0,4,6,1.50,1.50,1,1.2,1,0.6,1.50


In [144]:
res = playerStats.groupby(['playerId'])["sog"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['sog'])&(res['sog']<res['High'])].drop(columns=['Low', 'High'])

#toi
res = playerStats.groupby(['playerId'])["toi"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=ShotsNoOutliers.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['toi'])&(res['toi']<res['High'])].drop(columns=['Low', 'High'])


ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['sog'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).sog.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,CHI,8477987,Ryan,Donato,2025020001,0,1,16.00,5,5,0,5,12,2.4,2.4,1,2.4,1,1.2,2.4,2,2,1.0
1,CHI,8477450,Jason,Dickinson,2025020001,0,1,14.23,4,5,0,5,6,1.2,1.5,1,1.2,1,0.6,1.5,1,1,1.0
2,CHI,8482703,Colton,Dach,2025020001,0,1,13.48,5,5,0,5,4,0.8,0.8,1,0.8,1,0.4,0.8,2,2,1.0
4,CHI,8483493,Frank,Nazar,2025020001,0,1,20.23,5,5,0,5,8,1.6,1.6,1,1.6,1,0.8,1.6,2,2,1.5
5,CHI,8481624,Ilya,Mikheyev,2025020001,0,1,17.12,5,5,0,5,9,1.8,1.8,1,1.8,1,0.9,1.8,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,CGY,8474150,Mikael,Backlund,2025020060,0,1,19.15,1,5,0,5,6,1.2,1.2,1,1.2,1,0.6,1.2,1,2,1.0
444,CGY,8481068,Yegor,Sharangovich,2025020060,0,1,13.63,1,5,0,5,9,1.8,1.8,1,1.8,1,0.9,1.8,1,2,2.0
446,CGY,8484180,Samuel,Honzek,2025020060,0,2,13.00,1,5,0,5,6,1.2,2.0,1,1.2,1,0.6,2.0,1,1,2.0
448,CGY,8478397,Rasmus,Andersson,2025020060,0,2,23.38,1,5,0,5,10,2.0,2.0,1,2.0,1,1.0,2.0,1,2,2.0


In [145]:
Teams

,City,TeamName,TeamID
0,Colorado,Avalanche,COL
1,Carolina,Hurricanes,CAR
2,Dallas,Stars,DAL
3,Detroit,Red Wings,DET
4,Boston,Bruins,BOS
5,Montréal,Canadiens,MTL
6,Washington,Capitals,WSH
7,Vegas,Golden Knights,VGK
8,Florida,Panthers,FLA
9,Edmonton,Oilers,EDM


In [146]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,NYR,TOR
1,NSH,MTL
2,SEA,OTT
3,FLA,NJD
4,WPG,PHI
5,COL,CBJ
6,EDM,NYI
7,VAN,DAL
8,BOS,VGK
9,CAR,ANA


In [147]:
shotSummary=ShotsNoOutliers[['Team', 'playerId','firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots','LastTenAvgShots', 'NoOutliersAvgSOG','avgshotshomevaway']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['Team'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='Team', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])
todaysSkaters['HomeFlag'] = np.where(todaysSkaters['Team'].isin(TodaysGames['HomeTeam'].tolist()), 1, 0)

# jointhis=playerStats[['playerId','opponentAbbrev','homeRoadFlag','GPHomeAwayvsOpp','avgshotsvsOppAtLoc','totalshotsvsOppAtLoc']].rename(
#     columns={"opponentAbbrev": "Opponent","homeRoadFlag": "HomeFlag"})

#todaysSkaters=todaysSkaters.merge(jointhis,on=['playerId','Opponent','HomeFlag'],how='left')
# todaysSkaters=todaysSkaters.merge(playerStats[['playerId','homeRoadFlag','AvgShotsSznByLoc']].rename(
#     columns={"homeRoadFlag": "HomeFlag"}),on=['playerId','HomeFlag'],how='left').drop_duplicates()
# todaysSkaters=todaysSkaters.merge(playerStats[['playerId','opponentAbbrev','AvgShotsAgainstThisOpp']].rename(
#     columns={"opponentAbbrev": "Opponent"}),on=['playerId','Opponent'],how='left').drop_duplicates()

# todaysSkaters['avgshotsvsOppAtLoc']=np.where(todaysSkaters['avgshotsvsOppAtLoc'].isna(), 0, todaysSkaters['avgshotsvsOppAtLoc'])
# todaysSkaters['AvgShotsAgainstThisOpp']=np.where(todaysSkaters['AvgShotsAgainstThisOpp'].isna(), 0, todaysSkaters['AvgShotsAgainstThisOpp'])
# todaysSkaters['GPHomeAwayvsOpp']=np.where(todaysSkaters['GPHomeAwayvsOpp'].isna(), 0, todaysSkaters['GPHomeAwayvsOpp'])

# todaysSkaters=todaysSkaters[['Team',	'playerId',	'firstName',	'lastName',	'SeasonAvgSOG','AvgShotsSznByLoc',
#                              	'LastFiveAvgShots',	'LastTenAvgShots',	'NoOutliersAvgSOG',	'Opponent',
#                                     	'HomeFlag',	'GPHomeAwayvsOpp',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp']]

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,Team,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,avgshotshomevaway,Opponent,HomeFlag
0,TOR,8479318,Auston,Matthews,5.00,4.0,2.0,5.5,5.00,NYR,1
1,LAK,8477960,Adrian,Kempe,4.25,3.4,1.7,5.0,4.25,PIT,1
2,LAK,8477942,Kevin,Fiala,4.00,3.2,1.6,5.0,4.00,PIT,1
3,VGK,8478403,Jack,Eichel,4.50,3.6,1.8,5.0,4.50,BOS,1
4,ANA,8483445,Cutter,Gauthier,5.33,3.2,1.6,5.0,5.33,CAR,1
5,NYR,8482109,Alexis,Lafrenière,3.80,3.8,1.9,4.0,3.80,TOR,0
6,COL,8477492,Nathan,MacKinnon,3.75,3.0,1.5,4.0,3.75,CBJ,0
7,OTT,8482116,Tim,Stützle,4.00,3.2,1.6,4.0,4.00,SEA,1
8,COL,8480069,Cale,Makar,3.00,2.4,1.2,4.0,3.00,CBJ,0
9,ANA,8473986,Alex,Killorn,4.00,2.4,1.2,4.0,4.00,CAR,1


In [148]:
# goalieSTARTINGCSV2023=pd.read_csv('GoalieLastYear2023Stats.csv')  
# goalieSTARTINGCSV2024=pd.read_csv('GoalieThisYear2024Stats.csv')  
#goalieSTARTINGCSV2025=pd.read_csv('NHL2526GoalieStats.csv')  
goalieSTARTINGCSV2025=pd.read_csv('NewGoalieStats2526.csv')  

goalieSTARTINGCSV=goalieSTARTINGCSV2025#pd.concat([goalieSTARTINGCSV2023, goalieSTARTINGCSV2024])
#goalieSTARTINGCSV=goalieSTARTINGCSV[goalieSTARTINGCSV['gamesStarted']==1]
goalieSTARTINGCSV.sort_values(by=['gameId'])

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,firstName,lastName,Team,gameId,homeRoadFlag
0,8481519,30,{'default': 'S. Knight'},G,29/31,4/5,1/1,34/37,0.918919,2,1,0,0,3,59:04,True,L,37,34,Spencer,Knight,CHI,2025020001,0
1,8475683,72,"{'default': 'S. Bobrovsky', 'cs': 'S. Bobrovsk...",G,15/17,2/2,0/0,17/19,0.894737,2,0,0,0,2,60:00,True,W,19,17,Sergei,Bobrovsky,FLA,2025020001,1
2,8481668,37,"{'default': 'A. Silovs', 'cs': 'A. Šilovs', 's...",G,22/22,3/3,0/0,25/25,1.000000,0,0,0,0,0,60:00,True,W,25,25,Arturs,Silovs,PIT,2025020002,0
3,8478048,31,"{'default': 'I. Shesterkin', 'cs': 'I. Šesťork...",G,28/29,0/0,0/0,28/29,0.965517,1,0,0,0,1,59:37,True,L,29,28,Igor,Shesterkin,NYR,2025020002,1
4,8475809,41,{'default': 'S. Wedgewood'},G,17/17,7/8,0/0,24/25,0.960000,0,1,0,0,1,59:53,True,W,25,24,Scott,Wedgewood,COL,2025020003,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,8475660,39,{'default': 'C. Talbot'},G,17/18,3/3,0/0,20/21,0.952381,1,0,0,0,1,59:58,True,W,21,20,Cam,Talbot,DET,2025020058,1
116,8482821,40,"{'default': 'A. Soderblom', 'cs': 'A. Söderblo...",G,23/26,0/0,0/0,23/26,0.884615,3,0,0,0,3,60:00,True,W,26,23,Arvid,Soderblom,CHI,2025020059,0
117,8480981,30,{'default': 'J. Hofer'},G,13/19,2/3,0/0,15/22,0.681818,6,1,0,0,7,47:45,True,L,22,15,Joel,Hofer,STL,2025020059,1
118,8482445,1,{'default': 'D. Cooley'},G,19/21,9/9,1/1,29/31,0.935484,2,0,0,0,2,58:42,True,L,31,29,Devin,Cooley,CGY,2025020060,0


In [149]:
goalieStats=goalieSTARTINGCSV[['Team','playerId','firstName','lastName','gameId','starter','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['PlayerGameNumber'] = goalieStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
goalieStats['TeamGameNumber'] = goalieStats.groupby(['Team'])['gameId'].cumcount(ascending=False)+1
goalieStats['PlayerGP']=goalieStats.groupby(['playerId']).starter.transform(np.sum)
goalieStats['TeamGP']=goalieStats.groupby(['Team']).starter.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['Team']).shotsAgainst.transform(np.sum)/goalieStats['TeamGP']
goalieStats['RankAvgShotsAgainst']=round(goalieStats['AvgShotsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgGoalsAgainst']=round(goalieStats['AvgGoalsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgSavePctg']=round(goalieStats['AvgSavePctg'].rank(pct=True)*100,2)

goalieStats['LastFiveFlag'] = np.where(goalieStats['TeamGameNumber'] >= goalieStats['TeamGP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['Team','PlayerGP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['Team'])['PlayerGP'].rank(ascending=False).astype(int)
summaryGoalieStats=summaryGoalieStats[['Team','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','Team':'GoalieteamAbbrev'})
summaryGoalieStats.head()

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,CHI,1,Spencer,Knight,69.17,38.75,69.17
1,FLA,1,Sergei,Bobrovsky,11.67,22.08,66.25
2,PIT,1,Arturs,Silovs,63.75,52.08,52.08
3,NYR,1,Igor,Shesterkin,30.83,2.92,97.08
4,COL,1,Scott,Wedgewood,43.75,22.08,75.42


In [160]:
full=todaysSkaters.merge(summaryGoalieStats
                         [summaryGoalieStats['GPRank']==1]
                         , left_on='Opponent', right_on='GoalieteamAbbrev', how='left'
                         ).drop(columns=['GoalieteamAbbrev','GPRank'])
full['lastfivemargin']=full['LastFiveAvgShots']-full['NoOutliersAvgSOG']

full['locationmargin']=full['avgshotshomevaway']-full['NoOutliersAvgSOG']
full['combinedmargin']=full['lastfivemargin']+full['locationmargin']


full=full[['Team', 'HomeFlag', 'firstName', 'lastName', 'NoOutliersAvgSOG','lastfivemargin',
'avgshotshomevaway','locationmargin','combinedmargin',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'GoalieLastName',
       'RankAvgShotsAgainst','RankAvgGoalsAgainst', 'RankAvgSavePctg'
        ]].rename(columns={"NoOutliersAvgSOG": "ExpectedSOG"}).round(2)

full.sort_values(by=['ExpectedSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(25)
todaysdate=str(pd.Timestamp.now().date())
full.to_csv(todaysdate+'skaterstats.csv', index=False)

In [ ]:
# margin=full
# margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']


# margin=margin[['teamAbbrev', 'HomeFlag', 'firstName', 'lastName', 'ExpectedSOG','AvgShotsSznByLoc',
#        'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
#         'GoalieLastName',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp',
#         'SeasonAvgSOG', 'playerId', 
#        'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

# margin.sort_values(by=['lastfivemargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [155]:
# margin['locationmargin']=margin['AvgShotsSznByLoc']-margin['ExpectedSOG']

# margin.sort_values(by=['locationmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)


In [158]:
# margin['combinedmargin']=margin['lastfivemargin']+margin['locationmargin']

# margin[['firstName',	'lastName','teamAbbrev',	'HomeFlag','combinedmargin',
# 		'ExpectedSOG','LastFiveAvgShots',	'AvgShotsSznByLoc'	,
#         	'LastTenAvgShots',	'Opponent',	'RankAvgShotsAgainst',	'GoalieLastName',
#             			'SeasonAvgSOG']].sort_values(by=['combinedmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [ ]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df